# Exploring the LVIS Collections in MAAP

Authors: Rajat Shinde (UAH), Sheyenne Kirkland (UAH), Alex Mandel (DevSeed), Jamison French (DevSeed), Emile Tenezakis (DevSeed), Brian Freitag (NASA MSFC)

Date: August 17, 2023

Description: In this tutorial, we will explore the LVIS (Land Vegetation and Ice Sensor) based data products and work on accessing these data products using MAAP.

### Run This Notebook
To access and run this tutorial within MAAP's Algorithm Development Environment (ADE), please refer to the ["Getting started with the MAAP"](https://docs.maap-project.org/en/latest/getting_started/getting_started.html) section of our documentation.

Disclaimer: It is highly recommended to run a tutorial within MAAP's ADE, which already includes packages specific to MAAP, such as maap-py. Running the tutorial outside of the MAAP ADE may lead to errors.

### Importing Packages

We import the `os` module, import the `MAAP` package, and create a new MAAP class instance.

In [6]:
# import os module
import os

# import the MAAP package to handle queries
from maap.maap import MAAP

# import printing package to help display outputs
from pprint import pprint

# invoke the MAAP search client
maap = MAAP()

Unable to load config file from source maap.cfg 
Unable to load config file from source ./maap.cfg 
Unable to load config file from source /projects/maap.cfg 


In [7]:
# search all collections
results = maap.searchCollection(cmr_host="cmr.earthdata.nasa.gov")

# print the number of collections
pprint(f'Got {len(results)} results')

'Got 100 results'


The above cell searches for all the collections in the NASA CMR and stores the list in the `results` variable.

### Creating a Data Directory for this Tutorial

In [10]:
# set data directory path
dataDir = './LVIS_data'

# check if directory exists -> if directory doesn't exist, directory is created
if not os.path.exists(dataDir):
    os.mkdir(dataDir)

After executing the previous cell, we can observe that the data directory has been created and all the files in this tutorial will be downloaded to this directory.

## Accessing the LVIS Facility L1B Geolocated Return Energy Waveforms V001

### About the Dataset

The [LVIS Facility L1B Geolocated Return Energy Waveforms V001 dataset](https://doi.org/10.5067/XQJ8PN8FTIDG) contains Level-1B geolocated return energy waveforms collected by the NASA Land, Vegetation and Ice Sensor (LVIS) Facility, an imaging lidar and camera sensor suite. The short name for this collection is `LVISF1B`. This short name is used for searching this collection in the NASA CMR using the `searchCollection()` method.

### Searching the Collection

In [11]:
lvisf1b_collections = maap.searchCollection(
    short_name='LVISF1B',
    version='1',
    cmr_host='cmr.earthdata.nasa.gov'
)
lvisf1b_collections

[{'concept-id': 'C1723866745-NSIDC_ECS',
  'revision-id': '38',
  'format': 'application/echo10+xml',
  'Collection': {'ShortName': 'LVISF1B',
   'VersionId': '1',
   'InsertTime': '2023-08-17T08:16:35.265Z',
   'LastUpdate': '2023-08-17T08:16:35.265Z',
   'LongName': 'Not provided',
   'DataSetId': 'LVIS Facility L1B Geolocated Return Energy Waveforms V001',
   'Description': 'This data set contains Level-1B geolocated return energy waveforms collected by the NASA Land, Vegetation, and Ice Sensor (LVIS) Facility, an imaging lidar and camera sensor suite.',
   'DOI': {'DOI': '10.5067/XQJ8PN8FTIDG'},
   'StandardProduct': 'false',
   'RevisionDate': '2023-05-31T00:00:00.000Z',
   'SuggestedUsage': 'Scientific Research',
   'ProcessingCenter': 'NASA/GSFC/SED/ESD/LRSL',
   'ProcessingLevelId': 'Level 1B',
   'ProcessingLevelDescription': 'Sensor units',
   'ArchiveCenter': 'NASA NSIDC DAAC',
   'CollectionState': 'PLANNED',
   'RestrictionComment': ' These data are freely, openly, and ful

The above cell searches and generates the metadata associated with the LVISF1B collection. The `concept-id` in the above output is important and defines the collection id. Using this collection id (See `COLLECTION_ID` below), we will be searching for the granules in this collection.

### Searching and Downloading a Granule

We use `searchGranule()` method for searching granules in a particular collection. Once the list of granules is retrieved, we can download a granule to above-defined path by using the `getData()` method.

In [14]:
COLLECTION_ID = 'C1723866745-NSIDC_ECS'

results = maap.searchGranule(
    concept_id=COLLECTION_ID,
    cmr_host="cmr.earthdata.nasa.gov"
)
pprint(f'Got {len(results)} results')


#Validating download
download = results[0].getData(dataDir)
print(download)

'Got 20 results'
./LVIS_data/LVISF1B_US2018_1107_R2011_067463.h5


### Validating the Downloaded Product

The downloaded product is stored in a Hierarchical Data Format, Version 5 or HDF5 file format (`.h5`). Such files can be accessed in Python by using the [h5py](https://pypi.org/project/h5py/) library. 

In [15]:
#Testing the HDF5 files
filename = os.path.join(dataDir, "LVISF1B_US2018_1107_R2011_067463.h5")
import h5py
with h5py.File(filename, "r") as f: 
    print("Keys: %s" % f.keys())
    # get first object name/key; may or may NOT be a group
    a_group_key = list(f.keys())[0]

    # get the object type for a_group_key: usually group or dataset
    print(type(f[a_group_key])) 

    # If a_group_key is a group name, 
    # this gets the object names in the group and returns as a list
    data = list(f[a_group_key])

    # If a_group_key is a dataset name, 
    # this gets the dataset values and returns as a list
    data = list(f[a_group_key])
    # preferred methods to get dataset values:
    ds_obj = f[a_group_key]      # returns as a h5py dataset object
    ds_arr = f[a_group_key][()]  # returns as a numpy array
    
    print(ds_arr)

Keys: <KeysViewHDF5 ['AZIMUTH', 'INCIDENTANGLE', 'LAT0', 'LAT1215', 'LFID', 'LON0', 'LON1215', 'RANGE', 'RXWAVE', 'SHOTNUMBER', 'SIGMEAN', 'TIME', 'TXWAVE', 'Z0', 'Z1215', 'ancillary_data']>
<class 'h5py._hl.dataset.Dataset'>
[159.14177  159.4966   158.7841   ... 127.461716 124.07847  124.752396]


Since we are able to access data values from the downloaded granule, it is justified that the downloaded file is a valid data product.

## Acessing the ABoVE LVIS L1B Geolocated Return Energy Waveforms V001

### About the Dataset
The [ABoVE LVIS L1B Geolocated Return Energy Waveforms V001 dataset](https://doi.org/10.5067/UMRAWS57QAFU) contains return energy waveform data over Alaska and Western Canada measured by the NASA Land, Vegetation, and Ice Sensor (LVIS), an airborne lidar scanning laser altimeter. The data were collected as part of NASA's Terrestrial Ecology Program campaign, the Arctic-Boreal Vulnerability Experiment (ABoVE). The short name for this collection is `ABLVIS1B`.

### Searching the Collection

In [16]:
ablvis1b_collections = maap.searchCollection(
    short_name='ABLVIS1B',
    version='1',
    cmr_host='cmr.earthdata.nasa.gov'
)
ablvis1b_collections

[{'concept-id': 'C1513105920-NSIDC_ECS',
  'revision-id': '49',
  'format': 'application/echo10+xml',
  'Collection': {'ShortName': 'ABLVIS1B',
   'VersionId': '1',
   'InsertTime': '2023-08-17T08:13:15.578Z',
   'LastUpdate': '2023-08-17T08:13:15.578Z',
   'LongName': 'Not provided',
   'DataSetId': 'ABoVE LVIS L1B Geolocated Return Energy Waveforms V001',
   'Description': "This data set contains return energy waveform data over Alaska and Western Canada measured by the NASA Land, Vegetation, and Ice Sensor (LVIS), an airborne lidar scanning laser altimeter. The data were collected as part of NASA's Terrestrial Ecology Program campaign, the Arctic-Boreal Vulnerability Experiment (ABoVE).",
   'DOI': {'DOI': '10.5067/UMRAWS57QAFU'},
   'StandardProduct': 'false',
   'RevisionDate': '2023-07-05T00:00:00.000Z',
   'SuggestedUsage': 'Scientific Research',
   'ProcessingCenter': 'NASA/GSFC/SED/ESD/LRSL',
   'ProcessingLevelId': 'Level 1B',
   'ProcessingLevelDescription': 'Sensor units',


Based on the output of the above cell, we can retrieve the `concept-id` as collection id and use in the subsequent cells for downloading the granules from this collection.

### Searching and Downloading a Granule

In [31]:
COLLECTION_ID = 'C1513105920-NSIDC_ECS'

results = maap.searchGranule(
    concept_id=COLLECTION_ID,
    cmr_host="cmr.earthdata.nasa.gov"
)
pprint(f'Got {len(results)} results')


#Validating download
download = results[0].getData(dataDir)
print(download)

'Got 20 results'
./LVIS_data/LVIS1B_ABoVE2017_0629_R1803_056233.h5


### Validating the Downloaded Product

Similar to the `LVISF1B` dataset (as discussed above), `ABLVIS1B` dataset has products stored as `.h5` files. We will be following the same steps as described above to validate the downloaded product.

In [33]:
#Testing the HDF5 files
filename = os.path.join(dataDir, "LVIS1B_ABoVE2017_0629_R1803_056233.h5")
import h5py
with h5py.File(filename, "r") as f: 
    print("Keys: %s" % f.keys())
    # get first object name/key; may or may NOT be a group
    a_group_key = list(f.keys())[0]

    # get the object type for a_group_key: usually group or dataset
    print(type(f[a_group_key])) 

    # If a_group_key is a group name, 
    # this gets the object names in the group and returns as a list
    data = list(f[a_group_key])

    # If a_group_key is a dataset name, 
    # this gets the dataset values and returns as a list
    data = list(f[a_group_key])
    # preferred methods to get dataset values:
    ds_obj = f[a_group_key]      # returns as a h5py dataset object
    ds_arr = f[a_group_key][()]  # returns as a numpy array
    
    print(ds_arr)

Keys: <KeysViewHDF5 ['AZIMUTH', 'INCIDENTANGLE', 'LAT0', 'LAT1215', 'LFID', 'LON0', 'LON1215', 'RANGE', 'RXWAVE', 'SHOTNUMBER', 'SIGMEAN', 'TIME', 'TXWAVE', 'Z0', 'Z1215', 'ancillary_data']>
<class 'h5py._hl.dataset.Dataset'>
[209.55528 209.98833 209.2643  ... 181.83081 183.49066 182.28035]


## Accessing LVIS Facility L2 Geolocated Surface Elevation and Canopy Height Product V001

### About the Dataset
The [LVIS Facility L2 Geolocated Surface Elevation and Canopy Height Product V001 dataset](https://doi.org/10.5067/VP7J20HJQISD) contains Level-2 geolocated surface elevation and canopy height measurements collected by the NASA Land, Vegetation, and Ice Sensor (LVIS) Facility, an imaging lidar and camera sensor suite. The short name for this collection is `LVISF2`.

### Searching the Collection

In [37]:
lvisf2_collections = maap.searchCollection(
    short_name='LVISF2',
    version='1',
    cmr_host='cmr.earthdata.nasa.gov'
)
lvisf2_collections

[{'concept-id': 'C1723866830-NSIDC_ECS',
  'revision-id': '29',
  'format': 'application/echo10+xml',
  'Collection': {'ShortName': 'LVISF2',
   'VersionId': '1',
   'InsertTime': '2023-08-17T16:36:12.683Z',
   'LastUpdate': '2023-08-17T16:36:12.683Z',
   'LongName': 'Not provided',
   'DataSetId': 'LVIS Facility L2 Geolocated Surface Elevation and Canopy Height Product V001',
   'Description': 'This data set contains Level-2 geolocated surface elevation and canopy height measurements collected by the NASA Land, Vegetation, and Ice Sensor (LVIS) Facility, an imaging lidar and camera sensor suite.',
   'DOI': {'DOI': '10.5067/VP7J20HJQISD'},
   'StandardProduct': 'false',
   'RevisionDate': '2023-05-31T00:00:00.000Z',
   'SuggestedUsage': 'Scientific Research',
   'ProcessingCenter': 'NASA/GSFC/SED/ESD/LRSL',
   'ProcessingLevelId': 'Level 2',
   'ProcessingLevelDescription': 'Derived geophysical variables',
   'ArchiveCenter': 'NASA NSIDC DAAC',
   'CollectionState': 'PLANNED',
   'Res

### Searching and Downloading a Granule

In [61]:
COLLECTION_ID = 'C1723866830-NSIDC_ECS' # Collection title: "NASA Shuttle Radar Topography Mission Global 1 arc second V003"

results = maap.searchGranule(
    concept_id=COLLECTION_ID,
    cmr_host="cmr.earthdata.nasa.gov"
)
pprint(f'Got {len(results)} results')


#Validating download
download = results[0].getData(dataDir)
print(download)

We follow the same steps as above for searching and downloading a granule from the LVISF2 collection. However, as we observe, the downloaded product in this case is a `.txt` file. So, we will be using the [numpy](https://numpy.org/) Python package for accessing the downloaded text file.

### Validating the Downloaded Product

In [34]:
#Testing the TXT files from the LVIS collections
import numpy as np
data = np.genfromtxt(os.path.join(dataDir, "./LVISF2_US2018_1107_R2011_067463.TXT"))
print(data)

[[1.95842916e+09 5.93768900e+06 6.74631929e+04 ... 1.00000000e+00
  1.00000000e+00 1.00000000e+00]
 [1.95842916e+09 5.93769000e+06 6.74631931e+04 ... 1.00000000e+00
  1.00000000e+00 1.00000000e+00]
 [1.95842916e+09 5.93769100e+06 6.74631934e+04 ... 1.00000000e+00
  1.00000000e+00 1.00000000e+00]
 ...
 [1.95842916e+09 6.49713000e+06 6.76030543e+04 ... 1.00000000e+00
  1.00000000e+00 1.00000000e+00]
 [1.95842916e+09 6.49713100e+06 6.76030545e+04 ... 1.00000000e+00
  1.00000000e+00 1.00000000e+00]
 [1.95842916e+09 6.49713200e+06 6.76030548e+04 ... 1.00000000e+00
  1.00000000e+00 1.00000000e+00]]


As we can observe from the output of the above cell, the data values are accessible from the downloaded product. Hence, validating the downloaded file.

## Accessing the ABoVE LVIS L2 Geolocated Surface Elevation Product V001

### About the Dataset
This [dataset](https://doi.org/10.5067/IA5WAX7K3YGY) contains surface elevation data over Alaska and Western Canada measured by the NASA Land, Vegetation, and Ice Sensor (LVIS), an airborne lidar scanning laser altimeter. The data were collected as part of NASA's Terrestrial Ecology Program campaign, the Arctic-Boreal Vulnerability Experiment (ABoVE). The short name for this collection is `ABLVIS2`.

### Searching the Collection

In [38]:
ablvis2_collections = maap.searchCollection(
    short_name='ABLVIS2',
    version='1',
    cmr_host='cmr.earthdata.nasa.gov'
)
ablvis2_collections

[{'concept-id': 'C1513105984-NSIDC_ECS',
  'revision-id': '43',
  'format': 'application/echo10+xml',
  'Collection': {'ShortName': 'ABLVIS2',
   'VersionId': '1',
   'InsertTime': '2023-08-17T20:08:47.887Z',
   'LastUpdate': '2023-08-17T20:08:47.887Z',
   'LongName': 'Not provided',
   'DataSetId': 'ABoVE LVIS L2 Geolocated Surface Elevation Product V001',
   'Description': "This data set contains surface elevation data over Alaska and Western Canada measured by the NASA Land, Vegetation, and Ice Sensor (LVIS), an airborne lidar scanning laser altimeter. The data were collected as part of NASA's Terrestrial Ecology Program campaign, the Arctic-Boreal Vulnerability Experiment (ABoVE).",
   'DOI': {'DOI': '10.5067/IA5WAX7K3YGY'},
   'StandardProduct': 'false',
   'RevisionDate': '2023-05-31T00:00:00.000Z',
   'SuggestedUsage': 'Scientific Research',
   'ProcessingCenter': 'NASA/GSFC/SED/ESD/LRSL',
   'ProcessingLevelId': 'Level 2',
   'ProcessingLevelDescription': 'Derived geophysical v

### Searching and Downloading a Granule

We follow the same steps as above.

In [64]:
COLLECTION_ID = 'C1513105984-NSIDC_ECS'

results = maap.searchGranule(
    concept_id=COLLECTION_ID,
    cmr_host="cmr.earthdata.nasa.gov"
)
pprint(f'Got {len(results)} results')


#Validating download
download = results[0].getData(dataDir)
print(download)

### Validating the Dataset

Since the output for this data product is also downloaded as a text file, we will follow the same steps as decribed above for the `LVISF2` data product.

In [35]:
#Testing the downloaded TXT file
import numpy as np
data = np.genfromtxt(os.path.join(dataDir,"./LVIS2_ABoVE2017_0629_R1803_056233.TXT"))
print(data)

[[1.95793304e+09 3.99310900e+06 5.62334890e+04 ... 1.00000000e+00
  1.00000000e+00 1.00000000e+00]
 [1.95793304e+09 3.99311000e+06 5.62334890e+04 ... 1.00000000e+00
  1.00000000e+00 1.00000000e+00]
 [1.95793304e+09 3.99311100e+06 5.62334890e+04 ... 1.00000000e+00
  1.00000000e+00 1.00000000e+00]
 ...
 [1.95793306e+09 5.43227700e+06 5.65932850e+04 ... 1.00000000e+00
  1.00000000e+00 1.00000000e+00]
 [1.95793306e+09 5.43227900e+06 5.65932850e+04 ... 1.00000000e+00
  1.00000000e+00 1.00000000e+00]
 [1.95793306e+09 5.43228200e+06 5.65932860e+04 ... 1.00000000e+00
  1.00000000e+00 1.00000000e+00]]


As observed from the output of the above cell, the data values are accessible from the downloaded product. Hence, validating the downloaded file.

## Accessing the AfriSAR LVIS L1B Geolocated Return Energy Waveforms V001

### About the Dataset
The [AfriSAR LVIS L1B Geolocated Return Energy Waveforms V001 dataset](https://doi.org/10.5067/ED5IYGVTB50Z) contains return energy waveform data over Gabon, Africa. The measurements were taken by the NASA Land, Vegetation, and Ice Sensor (LVIS), an airborne lidar scanning laser altimeter. The data were collected as part of a NASA campaign, in collaboration with the European Space Agency (ESA) mission AfriSAR. This collection has `AFLVIS1B` as it's short name.

### Searching the Collection

In [39]:
aflvis1b_collections = maap.searchCollection(
    short_name='AFLVIS1B',
    version='1',
    cmr_host='cmr.earthdata.nasa.gov'
)
aflvis1b_collections

[{'concept-id': 'C1549378019-NSIDC_ECS',
  'revision-id': '43',
  'format': 'application/echo10+xml',
  'Collection': {'ShortName': 'AFLVIS1B',
   'VersionId': '1',
   'InsertTime': '2023-08-17T20:08:55.923Z',
   'LastUpdate': '2023-08-17T20:08:55.923Z',
   'LongName': 'Not provided',
   'DataSetId': 'AfriSAR LVIS L1B Geolocated Return Energy Waveforms V001',
   'Description': 'This data set contains return energy waveform data over Gabon, Africa. The measurements were taken by the NASA Land, Vegetation, and Ice Sensor (LVIS), an airborne lidar scanning laser altimeter. The data were collected as part of a NASA campaign, in collaboration with the European Space Agency (ESA) mission AfriSAR.',
   'DOI': {'DOI': '10.5067/ED5IYGVTB50Z'},
   'StandardProduct': 'false',
   'RevisionDate': '2023-05-31T00:00:00.000Z',
   'SuggestedUsage': 'Scientific Research',
   'ProcessingCenter': 'NASA/GSFC/SED/ESD/LRSL',
   'ProcessingLevelId': 'Level 1B',
   'ProcessingLevelDescription': 'Sensor units',

### Searching and Downloading a Granule

We will be following the same steps as described above for the other data products.

In [70]:
COLLECTION_ID = 'C1549378019-NSIDC_ECS' # Collection title: "NASA Shuttle Radar Topography Mission Global 1 arc second V003"

results = maap.searchGranule(
    concept_id=COLLECTION_ID,
    cmr_host="cmr.earthdata.nasa.gov"
)
pprint(f'Got {len(results)} results')


#Validating download
download = results[0].getData(dataDir)
print(download)

'Got 20 results'
./LVIS1B_Gabon2016_0220_R1808_038024.h5


### Validating the Product

We observe that the downloaded product is stored as a `.h5` file similar to the `ABLVIS1B` and `LVISF1B` data products discussed above. So, we will be using the `h5py` package to validate the dowloaded product.

In [36]:
#Testing the HDF5 files
filename = os.path.join(dataDir,"LVIS1B_Gabon2016_0220_R1808_038024.h5")
import h5py
with h5py.File(filename, "r") as f: 
    print("Keys: %s" % f.keys())
    # get first object name/key; may or may NOT be a group
    a_group_key = list(f.keys())[0]

    # get the object type for a_group_key: usually group or dataset
    print(type(f[a_group_key])) 

    # If a_group_key is a group name, 
    # this gets the object names in the group and returns as a list
    data = list(f[a_group_key])

    # If a_group_key is a dataset name, 
    # this gets the dataset values and returns as a list
    data = list(f[a_group_key])
    # preferred methods to get dataset values:
    ds_obj = f[a_group_key]      # returns as a h5py dataset object
    ds_arr = f[a_group_key][()]  # returns as a numpy array
    
    print(ds_arr)

Keys: <KeysViewHDF5 ['AZIMUTH', 'INCIDENTANGLE', 'LAT0', 'LAT1023', 'LFID', 'LON0', 'LON1023', 'RANGE', 'RXWAVE', 'SHOTNUMBER', 'SIGMEAN', 'TIME', 'TXWAVE', 'Z0', 'Z1023', 'ancillary_data']>
<class 'h5py._hl.dataset.Dataset'>
[170.88405 171.3952  179.33963 ... 178.35054 178.84067 179.31787]


As observed from the output of the above cell, the data values are retrieved from the downloaded product. Hence, validating the downloaded file.

### References
1. Blair, J. B. and M. Hofton. (2020). LVIS Facility L1B Geolocated Return Energy Waveforms, Version 1 [Data Set]. Boulder, Colorado USA. NASA National Snow and Ice Data Center Distributed Active Archive Center. https://doi.org/10.5067/XQJ8PN8FTIDG. Date Accessed 08-17-2023.
2. Blair, J. B. and M. Hofton. (2018). ABoVE LVIS L1B Geolocated Return Energy Waveforms, Version 1 [Data Set]. Boulder, Colorado USA. NASA National Snow and Ice Data Center Distributed Active Archive Center. https://doi.org/10.5067/UMRAWS57QAFU. Date Accessed 08-17-2023.
3. Blair, J. B. and M. Hofton. (2020). LVIS Facility L2 Geolocated Surface Elevation and Canopy Height Product, Version 1 [Data Set]. Boulder, Colorado USA. NASA National Snow and Ice Data Center Distributed Active Archive Center. https://doi.org/10.5067/VP7J20HJQISD. Date Accessed 08-17-2023.
4. Blair, J. B. and M. Hofton. (2018). ABoVE LVIS L2 Geolocated Surface Elevation Product, Version 1 [Data Set]. Boulder, Colorado USA. NASA National Snow and Ice Data Center Distributed Active Archive Center. https://doi.org/10.5067/IA5WAX7K3YGY. Date Accessed 08-17-2023.
5. Blair, J. B. and M. Hofton. (2018). AfriSAR LVIS L1B Geolocated Return Energy Waveforms, Version 1 [Data Set]. Boulder, Colorado USA. NASA National Snow and Ice Data Center Distributed Active Archive Center. https://doi.org/10.5067/ED5IYGVTB50Z. Date Accessed 08-17-2023.